In [1]:
# pip install rdflib

In [2]:
import pandas as pd
import numpy as np
import random
import csv
import os
from itertools import islice
import urllib.parse
from rdflib import Graph
from rdflib import Literal
from rdflib import Namespace
from rdflib.namespace import RDF, RDFS, FOAF, XSD, URIRef

In [3]:
URI = Namespace("http://SDM_lab_2.org/")
g = Graph()


In [4]:
csv_dir = '/Users/zzy13/Desktop/Classes_at_UPC/SDM_Semantic_data_management/Lab_2/Lab_doc/data/CSVs/' 

csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]

dataframes = {}
for csv_file in csv_files:
    file_path = os.path.join(csv_dir, csv_file)
    df_name = os.path.splitext(csv_file)[0].replace('-', '_')
    dataframes[df_name] = pd.read_csv(file_path)

for df_name in dataframes.keys():
    print(df_name)

paper_cite_paper
journal_in_year
proceeding_in_year
scopus_500
authors
paper_in_year
journal
sample
paper_in_proceedings
paper_has_keywords
author_write
year
paper_belong_to_journal
conference
proceeding
paper
paper_belong_to_proceeding
keywords
paper_presented_in_conference
conference_detail
author_review


### Author -- [Write] --> ResearchPaper
<!-- ### Author -- [CorrespondingAuthor] --/> ResearchPaper -->

In [5]:
dataframes['author_write'].head(3)

,author_id,paper_id,corresponding
0,57219571200,10.1002/asi.24855,True
1,35726817600,10.3390/s24041209,True
2,58897160900,10.3390/s24041209,False


In [6]:
g.add((URI.Author, URI.Write, URI.ResearchPaper)) 

def Write_process(df):
    for _, record in df.iterrows():
        # Create URIs
        Author_id_uri = URIRef(URI.Author + "_" + str(record['author_id']))
        ResearchPaper_id_uri = URIRef(URI.ResearchPaper + "_" + str(record['paper_id']))
        
        g.add((Author_id_uri, RDF.type, URI.Author))
        g.add((ResearchPaper_id_uri, RDF.type, URI.ResearchPaper))
        
        g.add((Author_id_uri, URI.Write, ResearchPaper_id_uri))
        

Write_process(dataframes['author_write'])


### ResearchPaper --[submitted] --> SubmittedPaper

In [7]:
dataframes['paper'].head(3)

,title,abstract,pages,DOI,link,year
0,Challenges posed by hijacked journals in Scopus,This study presents and explains the phenomeno...,395-422,10.1002/asi.24855,https://www.scopus.com/inward/record.uri?eid=2...,2023
1,Normalizing Large Scale Sensor-Based MWD Data:...,In the context of geo-infrastructures and spec...,-,10.3390/s24041209,https://www.scopus.com/inward/record.uri?eid=2...,2022
2,Instant learning based on deep neural network ...,Biometric-based identity verification systems ...,32099-32122,10.1007/s11042-023-16751-6,https://www.scopus.com/inward/record.uri?eid=2...,2024


In [8]:
g.add((URI.ResearchPaper, URI.Submitted, URI.SubmittedPaper)) 

g.add((URI.ResearchPaper, URI.PaperTitle, XSD.string)) 

g.add((URI.ResearchPaper, URI.PaperAbstract, XSD.string)) 

def Submitted_process(df):
    for _, record in df.iterrows():
        # Create URIs
        paper_id_uri = URIRef(URI.ResearchPaper + "_" + str(record['DOI']))
        submitted_paper_id_uri = URIRef(URI.SubmittedPaper + "_" + str(record['DOI']))
        
        g.add((submitted_paper_id_uri, RDF.type, URI.SubmittedPaper))

        # Add the relationship to the graph
        g.add((paper_id_uri, URI.Submitted, submitted_paper_id_uri))

        g.add((paper_id_uri, URI.PaperTitle, Literal(record['title'], datatype=XSD.string))) 
        g.add((paper_id_uri, URI.PaperAbstract, Literal(record['abstract'], datatype=XSD.string))) 
        
Submitted_process(dataframes['paper'])

### ResearchPaper --[Cites] --> ResearchPaper

In [9]:
dataframes['paper_cite_paper'].head(3)

,start_id,end_id
0,10.1002/asi.24855,10.4230/LIPIcs.ICDT.2024.23
1,10.1002/asi.24855,10.1109/SSI58917.2023.10387755
2,10.1002/asi.24855,10.4230/LIPIcs.ICDT.2023.14


In [10]:
g.add((URI.ResearchPaper, URI.Cites, URI.ResearchPaper))

def paper_cites(df):
    for _, record in df.iterrows():
        # Create URIs
        paper_id_uri = URIRef(URI.ResearchPaper + "_" + str(record['start_id']))
        cites_uri = URIRef(URI.ResearchPaper + "_" + str(record['end_id']))
        # Add the relationship to the graph
        g.add((paper_id_uri, URI.Cites, cites_uri))

paper_cites(dataframes['paper_cite_paper'])

### ResearchPaper --[HasTopics] --> Topics

In [11]:
dataframes['paper_has_keywords'].head(3)

,paper_id,keywords
0,10.3390/s24041209,filtering process
1,10.3390/s24041209,measurement while drilling (MWD)
2,10.3390/s24041209,normalizing index


In [12]:
g.add((URI.ResearchPaper, URI.HasTopics, URI.Topics))

def paper_topics(df):
    for _, record in df.iterrows():
        # Create URIs
        paper_id_uri = URIRef(URI.ResearchPaper + "_" + str(record['paper_id']))
        topics_uri = URIRef(URI.Topics + "_" + urllib.parse.quote(record['keywords']))
        # Add the relationship to the graph
        g.add((topics_uri, RDF.type, URI.Topics))
        g.add((paper_id_uri, URI.HasTopics, topics_uri))

paper_topics(dataframes['paper_has_keywords'])

### Conference, Workshop, Proceeding, RegularConference, Venue

In [13]:
dataframes['conference_detail'].head(5)

,name,year,city,proceeding_name,ConferenceType
0,"1st IEEE Colombian Caribbean Conference, C3 2023",2021,Barranquilla,"1st IEEE Colombian Caribbean Conference, C3 2023",RegularConference
1,2023 International Conference on Innovation an...,2023,"Virtual, Online",2023 International Conference on Innovation an...,RegularConference
2,20th Free and Open Source Software for Geospat...,2023,Prizren,20th Free and Open Source Software for Geospat...,RegularConference
3,3rd International Conference on Machine Learni...,2023,NaN,3rd International Conference on Machine Learni...,RegularConference
4,ACM ASIA Conference on Computer and Communicat...,2024,Melbourne,18th ACM ASIA Conference on Computer and Commu...,RegularConference


In [14]:
g.add((URI.Conference, URI.BelongTo, URI.Proceedings))
g.add((URI.Workshop, URI.WorkshopIn, URI.Proceedings))
g.add((URI.RegularConference, URI.ConIn, URI.Proceedings))
g.add((URI.Proceedings, URI.InVenue, URI.Venue))

def process(df):
    for _, record in df.iterrows():
        # create Conference URI
        conference_uri = URIRef(URI.Conference + "_" + urllib.parse.quote(str(record.name)))
        
        # Create conference type URI
        conference_type = record['ConferenceType']
        if conference_type == 'RegularConference':
            conference_type_uri = URI.RegularConference
        elif conference_type == 'Workshop':
            conference_type_uri = URI.Workshop
        else:
            conference_type_uri = URI.Conference
        
        # g.add((conference_uri, RDF.type, URI.Conference))
        g.add((conference_uri, RDF.type, conference_type_uri))
        
        proceeding_name_uri = URIRef(URI.Proceedings + "_" + urllib.parse.quote(str(record['proceeding_name'])))
        g.add((proceeding_name_uri, RDF.type, URI.Proceedings))

        g.add((conference_uri, URI.BelongTo, proceeding_name_uri))

        venue_name_uri = URIRef(URI.Venue + "_" + urllib.parse.quote(str(record['city'])))
        g.add((venue_name_uri, RDF.type, URI.Venue))
        g.add((proceeding_name_uri, URI.InVenue, venue_name_uri))

process(dataframes['conference_detail'])

### ResearchPaper --[IsInProceeding] --> Proceedings

In [15]:
dataframes['paper_in_proceedings'].head(3)

,start_id,end_id
0,10.1109/ICICIS56802.2023.10430275,2023 International Conference on Integration o...
1,10.1145/3592307.3592335,"6th International Conference on Electronics, C..."
2,10.1109/NEleX59773.2023.10421720,2023 IEEE International Conference on Next Gen...


In [16]:
g.add((URI.ResearchPaper, URI.IsInProceeding, URI.Proceedings))

def paper_in_proceeding(df):
    for _, record in df.iterrows():
        # Create URIs
        paper_id_uri = URIRef(URI.ResearchPaper + "_" + str(record['start_id']))
        proceeding_uri = URIRef(URI.Proceedings + "_" + urllib.parse.quote(str(record['end_id'])))
        # Add the relationship to the graph
        g.add((paper_id_uri, URI.IsInProceeding, proceeding_uri))

paper_in_proceeding(dataframes['paper_belong_to_proceeding'])

### ResearchPaper --[IsInJournal] --> Volumes
### Journal -- [OfJournal] --> Volumes

In [17]:
dataframes['paper_belong_to_journal'].head(3)

,DOI,journal_name,year,volume
0,10.1002/asi.24855,Journal of the Association for Information Sci...,2023,Journal of the Association for Information Sci...
1,10.3390/s24041209,Sensors,2022,Sensors2022
2,10.1007/s11042-023-16751-6,Multimedia Tools and Applications,2024,Multimedia Tools and Applications2024


In [18]:
g.add((URI.ResearchPaper, URI.IsInJournal, URI.Volumes))
g.add((URI.Journal, URI.OfJournal, URI.Volumes))

def paper_in_journal(df):
    for _, record in df.iterrows():
        # Create URIs
        paper_id_uri = URIRef(URI.ResearchPaper + "_" + str(record['DOI']))
        volume_uri = URIRef(URI.Volumes + "_" + urllib.parse.quote(str(record['volume'])))
        journal_uri = URIRef(URI.Journal + "_" + urllib.parse.quote(str(record['journal_name'])))
        
        g.add((volume_uri, RDF.type, URI.Volumes))
        g.add((journal_uri, RDF.type, URI.Journal))
        # Add the relationship to the graph
        g.add((paper_id_uri, URI.IsInJournal, volume_uri))
        g.add((journal_uri, URI.OfJournal, volume_uri))

paper_in_journal(dataframes['paper_belong_to_journal'])

### Reviewer --[WriteReview] --> SubmittedPaper
### ReviewText -- [ReviewOf] --> SubmittedPaper

In [19]:
dataframes['author_review'].head(3)

,paper_id,reviewer_id,review_content,accept_possibility
0,10.1002/asi.24855,55445000700,Next certain tend million country compare.,0.254872
1,10.1002/asi.24855,57946022500,Hold thus safe difficult focus.,0.466417
2,10.1002/asi.24855,25921550000,Drug fund sure political social.,0.122583


In [20]:
g.add((URI.Reviewer, URI.WriteReview, URI.SubmittedPaper))
g.add((URI.ReviewText, URI.ReviewOf, URI.SubmittedPaper))

def review_paper(df):
    for _, record in df.iterrows():
        # Create URIs
        reviewer_uri = URIRef(URI.Reviewer + "_" + str(record['reviewer_id']))
        review_text_uri = URIRef(URI.ReviewText + "_" + urllib.parse.quote(str(record['review_content'])))
        submittedpaper_uri = URIRef(URI.SubmittedPaper + "_" + str(record['paper_id']))
        
        g.add((reviewer_uri, RDF.type, URI.Reviewer))
        g.add((review_text_uri, RDF.type, URI.ReviewText))
        # Add the relationship to the graph
        g.add((reviewer_uri, URI.WriteReview, submittedpaper_uri))
        g.add((review_text_uri, URI.ReviewOf, submittedpaper_uri))

review_paper(dataframes['author_review'])

# Export graph

In [21]:
for stmt in islice(g, 5):
    print(stmt)

(rdflib.term.URIRef('http://SDM_lab_2.org/ResearchPaper_10.12694/scpe.v25i1.2296'), rdflib.term.URIRef('http://SDM_lab_2.org/HasTopics'), rdflib.term.URIRef('http://SDM_lab_2.org/Topics_Emotional%20characteristics'))
(rdflib.term.URIRef('http://SDM_lab_2.org/ResearchPaper_10.1145/3616712.3616791'), rdflib.term.URIRef('http://SDM_lab_2.org/Cites'), rdflib.term.URIRef('http://SDM_lab_2.org/ResearchPaper_10.48786/edbt.2023.74'))
(rdflib.term.URIRef('http://SDM_lab_2.org/ResearchPaper_10.3233/JIFS-223030'), rdflib.term.URIRef('http://SDM_lab_2.org/Cites'), rdflib.term.URIRef('http://SDM_lab_2.org/ResearchPaper_10.1109/JIOT.2023.3327738'))
(rdflib.term.URIRef('http://SDM_lab_2.org/ResearchPaper_10.1109/MITS.2023.3346842'), rdflib.term.URIRef('http://SDM_lab_2.org/Cites'), rdflib.term.URIRef('http://SDM_lab_2.org/ResearchPaper_10.1145/3584372.3588672'))
(rdflib.term.URIRef('http://SDM_lab_2.org/Author_57222360072'), rdflib.term.URIRef('http://SDM_lab_2.org/Write'), rdflib.term.URIRef('http:/

In [22]:
# Export as Turtle
g.serialize(destination='/Users/zzy13/Desktop/Classes_at_UPC/SDM_Semantic_data_management/Lab_2/Lab_doc/data/lab_abox.ttl', format='turtle')

<Graph identifier=Nf0635b866afd4b8cbe6e81d699e4e6d3 (<class 'rdflib.graph.Graph'>)>